In [1]:
# Check GPU type
print("ça marche !")
!pip install nvidia-smi

# nvidia-smi est un outil en ligne de commande fourni par NVIDIA pour interagir avec les GPU NVIDIA installés sur un système.
# pas nécessaire pour kaggle car il gère l'accès aux ressources GPU

ça marche !


In [ ]:
!pip install --upgrade pip

# Install ultralytics
!pip -q install  ultralytics

!pip install ipywidgets
!pip install jupyter

# !pip install --upgrade jupyter
# !pip install --upgrade ipywidgets

#Bibliothèque pour la modification des images
!pip install -U albumentations

#Bibliothèque pour le réequilibrage des classes des données
!pip install imbalanced-learn

!ultralytics.check()

!pip install ipywidgets
!pip install jupyter
!pip install pillow 

#ultralytics est une bibliothèque Python qui fournit des modèles de détection d'objets, de segmentation sémantique et d'autres tâches d'apprentissage profond.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.8 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:


In [ ]:
# Import libraries
import pandas as pd                                                # Manipulation de données sous forme de tableaux
import os                                                          # Interactions avec le système d'exploitation
from pathlib import Path                                           # Manipulation de chemins de fichiers
import shutil                                                      # Opérations de copie et déplacement de fichiers
from sklearn.model_selection import train_test_split               # Division d'un jeu de données en ensembles d'entraînement et de test
from tqdm.notebook import tqdm                                     # Affichage de barres de progression dans un notebook Jupyter
import cv2                                                         # Traitement d'images
import yaml                                                        # Lecture et écriture de fichiers de configuration au format YAML
import matplotlib.pyplot as plt                                    # Création de visualisations
from ultralytics import YOLO                                       # Utilisation de modèles de détection d'objets
import multiprocessing                                             # Calculs en parallèle
from kaggle_secrets import UserSecretsClient
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
from collections import Counter
import numpy as np
from skimage.exposure import match_histograms
from tqdm import tqdm
import concurrent.futures
from PIL import Image

In [ ]:
# Lister tout le contenu du répertoire
!ls /kaggle/input/malaria-data

# Path to where your data is stored
DATA_DIR = Path('/kaggle/input/malaria-data') # Définit le chemin vers le répertoire contenant les données

# Preview data files available
os.listdir(DATA_DIR) # Affiche la liste des fichiers disponibles dans le répertoire des données

In [ ]:
# Set up directoris for training a yolo model

# Images directories
DATASET_DIR = Path('datasets/dataset')
IMAGES_DIR = DATASET_DIR / 'images'
TRAIN_IMAGES_DIR = IMAGES_DIR / 'train'
VAL_IMAGES_DIR = IMAGES_DIR / 'val'
TEST_IMAGES_DIR = IMAGES_DIR / 'test'

VAL_IMAGES_0_DIR = VAL_IMAGES_DIR / 'class_0'
VAL_IMAGES_1_DIR = VAL_IMAGES_DIR / 'class_1'
TRAIN_IMAGES_0_DIR = TRAIN_IMAGES_DIR / 'class_0'
TRAIN_IMAGES_1_DIR = TRAIN_IMAGES_DIR / 'class_1'

# Labels directories
LABELS_DIR = DATASET_DIR / 'labels'
TRAIN_LABELS_DIR = LABELS_DIR / 'train'
VAL_LABELS_DIR = LABELS_DIR / 'val'
TEST_LABELS_DIR = LABELS_DIR / 'test'

TRAIN_LABELS_0_DIR = TRAIN_LABELS_DIR / 'class_0'
TRAIN_LABELS_1_DIR = TRAIN_LABELS_DIR / 'class_1'
VAL_LABELS_0_DIR = VAL_LABELS_DIR / 'class_0'
VAL_LABELS_1_DIR = VAL_LABELS_DIR / 'class_1'


In [ ]:
# Unzip images to 'images' dir
if (DATA_DIR/ 'images.zip').exists():
    shutil.unpack_archive(DATASET_DIR / 'images.zip', 'IMAGES_DIR')  # Dézippe le fichier 'images.zip' vers le répertoire 'IMAGES_DIR'
else:
    print("fichier déjà dézippé")             # Si le fichier est déjà dézippé, affiche un message informant que le fichier est déjà extrait

In [ ]:
# Load train and test files
train = pd.read_csv(DATA_DIR / 'Train.csv')          # Charge le fichier CSV contenant les données d'entraînement
test = pd.read_csv(DATA_DIR / 'Test.csv')            # Charge le fichier CSV contenant les données de test
ss = pd.read_csv(DATA_DIR / 'SampleSubmission.csv')  # Charge le fichier CSV contenant un exemple de soumission

# Add an image_path column
train['image_path'] = [Path('../input/malaria-data/images/' + x) for x in train.Image_ID]   # Ajoute une colonne 'image_path' avec le chemin des images d'entraînement
test['image_path'] = [Path('../input/malaria-data/images/' + x) for x in test.Image_ID]    # Ajoute une colonne 'image_path' avec le chemin des images de test

# Map str classes to ints (label encoding targets)
train['class_id'] = train['class'].map({'Trophozoite': 0, 'WBC': 1, 'NEG': 2}) # Encode les classes sous forme d'entiers (0, 1, 2)

# Preview the head of the train set
train.head()            # Affiche les premières lignes du jeu de données d'entraînement

In [ ]:
# Filter out NEG class (Negative images - this images have no Trophozoite nor WBC in them)
# Yolo reccomends that when creating labels for negative images, you leave them empty
# https://github.com/ultralytics/yolov5/issues/3218
# As this is a starter nb, we will ignore the negative images
train = train[~(train['class'] == 'NEG')].reset_index(drop = True)       # Filtre le jeu de données d'entraînement pour ne garder que les images 
                                                                         # qui ne sont pas de classe 'NEG' (négatif), 
                                                                        # puis réinitialise l'index des lignes

In [ ]:
# Split data into training and validation
train_unique_imgs_df = train.drop_duplicates(subset = ['Image_ID'], ignore_index = True) # Crée un DataFrame contenant les images d'entraînement uniques
X_train, X_val = train_test_split(train_unique_imgs_df, test_size = 0.25, stratify=train_unique_imgs_df['class'], random_state=42) # Divise les images d'entraînement uniques en ensembles d'entraînement et de validation, 
                                                                                                                                   # en respectant la stratification des classes et en utilisant un seed aléatoire


# Cette syntaxe est utilisée pour le filtrage en pandas. 
# Elle sélectionne toutes les lignes du DataFrame train où le masque booléen est True.    
X_train = train[train.Image_ID.isin(X_train.Image_ID)]
X_val = train[train.Image_ID.isin(X_val.Image_ID)]

# Check shapes of training and validation data
X_train.shape, X_val.shape 

In [ ]:
X_train.head()

In [ ]:


# def match_histogram(image, reference):
#     # Convertir les images BGR en RGB (OpenCV utilise BGR par défaut)
#     image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     reference_rgb = cv2.cvtColor(reference, cv2.COLOR_BGR2RGB)
    
#     # Appliquer la correspondance d'histogramme
#     matched = match_histograms(image_rgb, reference_rgb, multichannel=True)
    
#     # Reconvertir l'image correspondante en BGR pour OpenCV
#     matched_bgr = cv2.cvtColor((matched * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)
#     return matched_bgr

# def batch_match_histograms(input_folder, output_folder, reference_image_path):
#     os.makedirs(output_folder, exist_ok=True)
    
#     # Charger l'image de référence
#     reference_image = cv2.imread(reference_image_path)
    
#     # Obtenir la liste des fichiers d'images
#     image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg') or f.endswith('.png')]
    
#     for image_file in tqdm(image_files, desc="Normalisation des images par correspondance d'histogramme"):
#         input_path = os.path.join(input_folder, image_file)
#         output_path = os.path.join(output_folder, image_file)
        
#         # Charger l'image actuelle
#         image = cv2.imread(input_path)
        
#         # Appliquer la correspondance d'histogramme
#         normalized_image = match_histogram(image, reference_image)
        
#         # Sauvegarder l'image normalisée
#         cv2.imwrite(output_path, normalized_image)

# # Utilisation
# input_folder = '../input/images/'  # Chemin vers les images d'origine
# output_folder = '../input/images/'  # Chemin vers les images normalisées
# reference_image_path = '../input/images/id_0051lznyrt.jpg'  # Chemin vers l'image de référence

# batch_match_histograms(input_folder, output_folder, reference_image_path)
X_val.head()

In [ ]:
# Preview target distribution, seems there a class imbalance that needs to be handled
# Affiche la répartition en pourcentage des classes dans les jeux de données d'entraînement et de validation
X_train['class'].value_counts(normalize = True), X_val['class'].value_counts(normalize = True)

In [ ]:
X_train.head()

In [ ]:
# Check if dirs exist, if they do, remove them, otherwise create them.
# This only needs to run once
for DIR in [TRAIN_IMAGES_DIR,VAL_IMAGES_DIR, TEST_IMAGES_DIR, TRAIN_LABELS_DIR,VAL_LABELS_DIR,TEST_LABELS_DIR,TRAIN_IMAGES_0_DIR,TRAIN_IMAGES_1_DIR,TRAIN_LABELS_0_DIR,TRAIN_LABELS_1_DIR,VAL_IMAGES_0_DIR,VAL_IMAGES_1_DIR,VAL_LABELS_0_DIR,VAL_LABELS_1_DIR]:
  if DIR.exists():
    shutil.rmtree(DIR)   # Si le répertoire existe, le supprime de manière récursive
  DIR.mkdir(parents=True, exist_ok = True)   # Crée le répertoire s'il n'existe pas, avec création des répertoires parents si nécessaire

In [ ]:
import shutil
from pathlib import Path
from tqdm import tqdm

# Définir les répertoires de destination pour chaque classe dans le dossier d'entraînement
CLASS_0_DIR_TRAIN = TRAIN_IMAGES_0_DIR
CLASS_1_DIR_TRAIN = TRAIN_IMAGES_1_DIR

# Définir les répertoires de destination pour chaque classe dans le dossier de validation
CLASS_0_DIR_VAL = VAL_IMAGES_0_DIR
CLASS_1_DIR_VAL = VAL_IMAGES_1_DIR

# Définir le répertoire de destination pour les images de test
TEST_IMAGES_DIR = Path(TEST_IMAGES_DIR)

# Créer tous les répertoires si ce n'est pas déjà fait
CLASS_0_DIR_TRAIN.mkdir(parents=True, exist_ok=True)
CLASS_1_DIR_TRAIN.mkdir(parents=True, exist_ok=True)
CLASS_0_DIR_VAL.mkdir(parents=True, exist_ok=True)
CLASS_1_DIR_VAL.mkdir(parents=True, exist_ok=True)
TEST_IMAGES_DIR.mkdir(parents=True, exist_ok=True)

# Copier les images d'entraînement dans les répertoires de classes correspondants
for _, row in tqdm(X_train.iterrows(), total=X_train.shape[0]):
    img_path = Path(row['image_path'])  # Chemin de l'image
    class_id = row['class_id']  # Classe de l'image

    # Copier dans le dossier de classe approprié
    if class_id == 0:
        shutil.copy(img_path, CLASS_0_DIR_TRAIN / img_path.name)
    elif class_id == 1:
        shutil.copy(img_path, CLASS_1_DIR_TRAIN / img_path.name)

# Copier les images de validation dans les répertoires de classes correspondants
for _, row in tqdm(X_val.iterrows(), total=X_val.shape[0]):
    img_path = Path(row['image_path'])
    class_id = row['class_id']

    # Copier dans le dossier de classe approprié
    if class_id == 0:
        shutil.copy(img_path, CLASS_0_DIR_VAL / img_path.name)
    elif class_id == 1:
        shutil.copy(img_path, CLASS_1_DIR_VAL / img_path.name)

# Copier les images de test dans le répertoire de test
for img in tqdm(test.image_path.unique()):
    img_path = Path(img)
    shutil.copy(img_path, TEST_IMAGES_DIR / img_path.name)


In [ ]:

# def convert_to_black_on_white(image):
#     # Convertir l'image en niveaux de gris
#     gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
#     # Inverser les couleurs pour avoir un fond blanc et des objets noirs
#     inverted_image = cv2.bitwise_not(gray_image)
    
#     # Convertir en BGR pour rester compatible avec d'autres opérations éventuelles
#     bgr_image = cv2.cvtColor(inverted_image, cv2.COLOR_GRAY2BGR)
    
#     return bgr_image

# def batch_process_images(input_folder, output_folder):
#     os.makedirs(output_folder, exist_ok=True)
    
#     # Obtenir la liste des fichiers d'images
#     image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg') or f.endswith('.png')]
    
#     # Vérifier que la liste des images n'est pas vide
#     if not image_files:
#         print(f"Aucune image trouvée dans le dossier {input_folder}")
#         return
   
#     for image_file in tqdm(image_files, desc="Conversion des images en noir sur blanc"):
#         input_path = os.path.join(input_folder, image_file)
#         output_path = os.path.join(output_folder, image_file)
        
#         # Charger l'image actuelle
#         image = cv2.imread(input_path)
#         if image is None:
#             print(f"Image non trouvée ou non lisible : {input_path}")
#             continue
        
#         # Convertir en noir sur blanc
#         bw_image = convert_to_black_on_white(image)
        
#         # Sauvegarder l'image convertie
#         success = cv2.imwrite(output_path, bw_image)
#         if not success:
#             print(f"Erreur lors de la sauvegarde de l'image : {output_path}")

# # Utilisation
# batch_process_images(TRAIN_IMAGES_DIR, TRAIN_IMAGES_DIR)
# batch_process_images(VAL_IMAGES_DIR, VAL_IMAGES_DIR)
# batch_process_images(TEST_IMAGES_DIR, TEST_IMAGES_DIR)
X_train.head()

In [ ]:

# import cv2
# import os
# import numpy as np
# from tqdm import tqdm
# from skimage.exposure import match_histograms

# def match_histogram(image, reference):
#     # Convertir les images BGR en RGB (OpenCV utilise BGR par défaut)
#     image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     reference_rgb = cv2.cvtColor(reference, cv2.COLOR_BGR2RGB)
    
#     # Appliquer la correspondance d'histogramme avec channel_axis au lieu de multichannel
#     matched = match_histograms(image_rgb, reference_rgb, channel_axis=-1)
    
#     # Reconvertir l'image correspondante en BGR pour OpenCV
#     matched_bgr = cv2.cvtColor((matched * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)
#     return matched_bgr

# def batch_match_histograms(input_folder, output_folder, reference_image_path):
#     os.makedirs(output_folder, exist_ok=True)
    
#     # Charger l'image de référence
#     reference_image = cv2.imread(reference_image_path)
#     if reference_image is None:
#         raise ValueError(f"Erreur : L'image de référence n'a pas été trouvée au chemin {reference_image_path}")
    
#     # print(f"Image de référence chargée avec succès : {reference_image_path}")

#     # Obtenir la liste des fichiers d'images
#     image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg') or f.endswith('.png')]
#     # Vérifier que la liste des images n'est pas vide
#     if not image_files:
#         print(f"Aucune image trouvée dans le dossier {input_folder}")
#         return
   
#     for image_file in tqdm(image_files, desc="Normalisation des images par correspondance d'histogramme"):
#         input_path = os.path.join(input_folder, image_file)
#         output_path = os.path.join(output_folder, image_file)
        
#         # Charger l'image actuelle
#         image = cv2.imread(input_path)
#         if image is None:
#             print(f"Image non trouvée ou non lisible : {input_path}")
#             continue
        
#         # Appliquer la correspondance d'histogramme
#         normalized_image = match_histogram(image, reference_image)
        
#         # Sauvegarder l'image normalisée
#         success = cv2.imwrite(output_path, normalized_image)
#         # if success:
#         #     print(f"Image normalisée sauvegardée : {output_path}")
#         # else:
#         #     print(f"Erreur lors de la sauvegarde de l'image : {output_path}")
        
#         # # Supprimer l'image originale
#         # try:
#         #     os.remove(input_path)
#         #     print(f"Image originale supprimée : {input_path}")
#         # except Exception as e:
#         #     print(f"Erreur lors de la suppression de l'image originale {input_path}: {e}")

# # Utilisation
# reference_image_path = '../input/id_2fzsen7crd.jpg'  # Chemin vers l'image de référence

# batch_match_histograms(TRAIN_IMAGES_DIR, TRAIN_IMAGES_DIR, reference_image_path)
# batch_match_histograms(VAL_IMAGES_DIR, VAL_IMAGES_DIR, reference_image_path)
# batch_match_histograms(TEST_IMAGES_DIR, TEST_IMAGES_DIR, reference_image_path)

#--------------------------------------------------------------------------------

# import torch
# import cv2
# import numpy as np
# import pandas as pd
# import albumentations as A
# import os
# from tqdm import tqdm
# import matplotlib.pyplot as plt

# # Répertoire de sauvegarde des images augmentées
# AUGMENTED_IMAGES_DIR = TRAIN_IMAGES_DIR
# os.makedirs(AUGMENTED_IMAGES_DIR, exist_ok=True)  # Crée le dossier si non existant

# # Fonction pour ajuster la luminosité à une valeur fixe
# def set_fixed_brightness(image, brightness_value=100):
#     image = np.clip(image + brightness_value, 0, 255).astype(np.uint8)
#     return image

# # Fonction pour ajuster l'exposition (similaire à la luminosité)
# def set_fixed_exposure(image, exposure_value=100):
#     image = np.clip(image * (1 + exposure_value / 100), 0, 255).astype(np.uint8)
#     return image

# # Enveloppe de luminosité et exposition fixes
# def fixed_brightness_and_exposure(image, **kwargs):
#     image = set_fixed_brightness(image, brightness_value=100)
#     return set_fixed_exposure(image, exposure_value=100)

# # Définir les augmentations
# transform = A.Compose([
#     A.RandomRotate90(),
#     A.HorizontalFlip(p=0.5),
#     A.VerticalFlip(p=0.5),
#     A.HueSaturationValue(hue_shift_limit=0, sat_shift_limit=(-100, 100), val_shift_limit=(0, 0), p=1.0),
#     A.Lambda(image=fixed_brightness_and_exposure),  # Appliquer luminosité et exposition fixes
#     A.RandomBrightnessContrast(p=0.5),  # Ajuster contraste et luminosité de manière aléatoire
#     A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # Ajouter du bruit
#     #A.Resize(640, 640),
# ])

# def load_and_augment_image(image_path):
#     # Vérifier si l'image existe
#     if not os.path.exists(image_path):
#         print(f"Image d'origine introuvable : {image_path}")
#         return None
    
#     # Charger l'image
#     image = cv2.imread(image_path)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convertir BGR en RGB
#     image = image.astype(np.uint8)  # S'assurer que l'image est en uint8 pour Albumentations

#     # Appliquer les augmentations
#     augmented = transform(image=image)
#     augmented_image = augmented['image']
    
#     return augmented_image

# # Fonction pour augmenter le dataset
# def augment_dataset(X_train):
#     # Compter le nombre d'instances pour chaque class_id
#     class_counts = X_train['class_id'].value_counts()

#     # Trouver la classe minoritaire et majoritaire
#     class_minority = class_counts.idxmin()
#     class_majority = class_counts.idxmax()
#     count_minority = class_counts.min()
#     count_majority = class_counts.max()

#     # Augmenter la classe minoritaire
#     augmented_images = []
#     augmented_records = []

#     while class_counts[class_minority] < count_majority:
#         for index, row in X_train.iterrows():
#             if row['class_id'] == class_minority:
#                 # Charger et augmenter l'image
#                 augmented_image = load_and_augment_image(row['image_path'])
                
#                 if augmented_image is not None:
#                     # Générer un nom unique pour l'image augmentée
#                     filename, file_extension = os.path.splitext(os.path.basename(row['image_path']))
#                     augmented_image_path = os.path.join(AUGMENTED_IMAGES_DIR, f"{filename}_aug{len(augmented_images)}{file_extension}")

#                     # Enregistrer l'image augmentée
#                     cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))
                    
# #                     # Vérifier que l'image est bien sauvegardée
# #                     if os.path.exists(augmented_image_path):
# #                         print(f"Image augmentée sauvegardée : {augmented_image_path}")
# #                     else:
# #                         print(f"Échec de la sauvegarde pour l'image : {augmented_image_path}")

#                     # Créer un enregistrement pour le DataFrame augmenté
#                     augmented_record = row.copy()
#                     augmented_record['Image_ID'] = f"{filename}_aug{len(augmented_images)}{file_extension}"
#                     augmented_record['image_path'] = augmented_image_path
#                     augmented_records.append(augmented_record)

#                     # Mettre à jour le compteur pour la classe minoritaire
#                     class_counts[class_minority] += 1
#                     if class_counts[class_minority] >= count_majority:
#                         break

#     # Créer un DataFrame pour les images augmentées
#     augmented_df = pd.DataFrame(augmented_records)
    
#     augmented_df.head()
#     # Concaténer le DataFrame original avec le DataFrame des augmentations
#     X_train = pd.concat([X_train, augmented_df], ignore_index=True)

#     return X_train

# # Charger le DataFrame et effectuer l'augmentation
# X_train = augment_dataset(X_train)

# # Afficher quelques informations après l'augmentation
# print(f"Nombre total d'images dans le DataFrame après augmentation : {len(X_train)}")
# print(f"Images augmentées dans le DataFrame : {X_train.tail(10)[['Image_ID', 'image_path']]}")

# # Vérification visuelle des images augmentées
# print("Affichage de quelques images augmentées :")
# for img_path in os.listdir(AUGMENTED_IMAGES_DIR)[:5]:  # Affiche seulement les 5 premières images augmentées pour validation
#     full_path = os.path.join(AUGMENTED_IMAGES_DIR, img_path)
#     if os.path.exists(full_path):
#         img = cv2.imread(full_path)
#         plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#         plt.title(f"Image augmentée: {img_path}")
#         plt.show()
#     else:
#         print(f"Image augmentée introuvable : {full_path}")

In [ ]:
import os
from PIL import Image
import multiprocessing
from tqdm import tqdm

# Fonction pour récupérer les dimensions d'une image
def get_image_dimensions(image_path):
    try:
        with Image.open(image_path) as img:
            return img.width, img.height
    except Exception as e:
        print(f"Erreur lors de l'ouverture de l'image {image_path}: {e}")
        return None, None

# Fonction pour créer une annotation YOLO pour une image spécifique
def save_yolo_annotation(row):
    image_id = row['Image_ID']
    image_path = row['image_path']
    output_dir = row['output_dir']
    
    # Récupérer les dimensions de l'image
    image_width, image_height = get_image_dimensions(image_path)
    
    # Vérifier si l'image a bien été chargée
    if image_width is None or image_height is None:
        return f"Erreur: Impossible d'ouvrir l'image {image_path}"

    # Définir le chemin du fichier d'annotation
    annotation_file_path = os.path.join(output_dir, f"{image_id.split('.')[0]}.txt")
    os.makedirs(output_dir, exist_ok=True)  # S'assurer que le dossier existe
    
    # Calcul des coordonnées normalisées (format YOLO)
    try:
        x_center = (row['xmin'] + row['xmax']) / 2 / image_width
        y_center = (row['ymin'] + row['ymax']) / 2 / image_height
        width = (row['xmax'] - row['xmin']) / image_width
        height = (row['ymax'] - row['ymin']) / image_height
        
        # Sauvegarder l'annotation dans le fichier
        with open(annotation_file_path, 'a') as f:
            f.write(f"{row['class_id']} {x_center} {y_center} {width} {height}\n")
        
        return f"Annotation sauvegardée dans {annotation_file_path}"
    
    except Exception as e:
        return f"Erreur lors de la sauvegarde de l'annotation pour {image_id}: {e}"

# Fonction pour paralléliser la création des annotations pour un DataFrame
def create_annotations(df, base_output_annotation_folder):
    # Utilisation de multiprocessing pour traiter chaque ligne du DataFrame en parallèle
    with multiprocessing.Pool() as pool:
        results = []
        for _, row in tqdm(df.iterrows(), total=df.shape[0]):
            # Déterminer le répertoire de sortie en fonction de la classe
            class_id = row['class_id']
            class_dir = f'class_{class_id}'
            output_annotation_folder = os.path.join(base_output_annotation_folder, class_dir)
            
            # Ajouter cette information au DataFrame
            row['output_dir'] = output_annotation_folder
            
            # Appeler la fonction de création des annotations
            results.append(pool.apply_async(save_yolo_annotation, (row,)))
        
        # Attendre la fin du traitement
        for result in results:
            result.get()  # Récupérer les résultats pour s'assurer que tout est traité

# Créer les annotations pour les ensembles d'entraînement et de validation
create_annotations(X_train, TRAIN_LABELS_DIR)
create_annotations(X_val, VAL_LABELS_DIR)


In [ ]:
# import os
# from PIL import Image
# import multiprocessing
# from tqdm import tqdm

# # Fonction pour récupérer les dimensions d'une image
# def get_image_dimensions(image_path):
#     try:
#         with Image.open(image_path) as img:
#             return img.width, img.height
#     except Exception as e:
#         print(f"Erreur lors de l'ouverture de l'image {image_path}: {e}")
#         return None, None

# # Fonction pour créer une annotation YOLO pour une image spécifique
# def save_yolo_annotation(row):
#     image_id = row['Image_ID']
#     image_path = row['image_path']
#     output_dir = row['output_dir']
    
#     # Récupérer les dimensions de l'image
#     image_width, image_height = get_image_dimensions(image_path)
    
#     # Vérifier si l'image a bien été chargée
#     if image_width is None or image_height is None:
#         return f"Erreur: Impossible d'ouvrir l'image {image_path}"

#     # Définir le chemin du fichier d'annotation
#     annotation_file_path = os.path.join(output_dir, f"{image_id.split('.')[0]}.txt")
#     os.makedirs(output_dir, exist_ok=True)  # S'assurer que le dossier existe
    
#     # Calcul des coordonnées normalisées (format YOLO)
#     try:
#         x_center = (row['xmin'] + row['xmax']) / 2 / image_width
#         y_center = (row['ymin'] + row['ymax']) / 2 / image_height
#         width = (row['xmax'] - row['xmin']) / image_width
#         height = (row['ymax'] - row['ymin']) / image_height
        
#         # Sauvegarder l'annotation dans le fichier
#         with open(annotation_file_path, 'a') as f:
#             f.write(f"{row['class_id']} {x_center} {y_center} {width} {height}\n")
        
#         return f"Annotation sauvegardée dans {annotation_file_path}"
    
#     except Exception as e:
#         return f"Erreur lors de la sauvegarde de l'annotation pour {image_id}: {e}"

# # Fonction pour paralléliser la création des annotations pour un DataFrame
# def create_annotations(df, output_annotation_folder):
#     # Ajoute une colonne 'output_dir' au DataFrame
#     df['output_dir'] = output_annotation_folder
    
#     # Utilisation de multiprocessing pour traiter chaque ligne du DataFrame en parallèle
#     with multiprocessing.Pool() as pool:
#         results = list(tqdm(pool.imap(save_yolo_annotation, df.to_dict('records')), total=len(df)))
    
#     # # Affichage des résultats
#     # for result in results:
#     #     print(result)

# # Créer les annotations pour les ensembles d'entraînement et de validation
# create_annotations(X_train, TRAIN_LABELS_DIR)
# create_annotations(X_val, VAL_LABELS_DIR)


In [ ]:
#normalisation des images avec pytorch
# import torch
# from torchvision import datasets, transforms
# import torch.utils.data

# # Fonction pour calculer la moyenne et l'écart-type
# def get_mean_std(loader):
#     num_pixels = 0
#     mean = 0.0
#     std = 0.0
#     for images, _ in loader:
#         batch_size, num_channels, height, width = images.shape
#         num_pixels += batch_size * height * width
#         mean += images.mean(axis=(0, 2, 3)).sum()
#         std += images.std(axis=(0, 2, 3)).sum()

#     mean /= num_pixels
#     std /= num_pixels
#     return mean, std

# # Charger ton jeu de données d'entraînement avec redimensionnement et transformation
# data_dir = TRAIN_IMAGES_DIR
# transform = transforms.Compose([
#     transforms.Resize((640, 640)),  # Redimensionner les images à 256x256
#     transforms.ToTensor()
# ])
# dataset = datasets.ImageFolder(data_dir, transform=transform)

# # Créer un DataLoader pour parcourir les images
# batch_size = 32
# loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# # Calculer la moyenne et l'écart-type
# mean, std = get_mean_std(loader)
# print(f"Mean: {mean}, Std: {std}")


In [ ]:
# from torchvision import transforms

# # Appliquer la normalisation avec la moyenne et l'écart-type calculées
# data_transforms = transforms.Compose([
#     transforms.Resize((640, 640)),  # Redimensionner l'image à une taille fixe pour YOLO
#     transforms.ToTensor(),
#     transforms.Normalize(mean=mean, std=std)  # Normaliser
# ])

# # Charger l'ensemble de données avec transformation
# train_dataset = datasets.ImageFolder(data_dir, transform=data_transforms)
# val_dataset = datasets.ImageFolder(VAL_IMAGES_DIR, transform=data_transforms)

# # DataLoader pour entraîner et valider le modèle
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
# val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)


In [ ]:
import cv2
import os
from pathlib import Path
import numpy as np


def normalize_image_for_yolo(image, size=(416, 416)):
    """
    Normalise une image pour un modèle YOLO en utilisant cv2.dnn.blobFromImage.
    """
    blob = cv2.dnn.blobFromImage(image, scalefactor=1/255.0, size=size, swapRB=True, crop=False)
    normalized_image = blob[0]  # Extraire le premier (et unique) blob
    return normalized_image

def process_images_in_directory(input_dir, output_size=(416, 416)):
    normalized_images = []
    for image_file in input_dir.glob("*.jpg"):
        image = cv2.imread(str(image_file))
        if image is not None:
            normalized_image = normalize_image_for_yolo(image, size=output_size)
            normalized_images.append(normalized_image)
        else:
            print(f"Image non trouvée ou non lisible : {image_file}")
    return normalized_images

# Traiter les images d'entraînement et de validation
train_images_normalized = process_images_in_directory(TRAIN_IMAGES_DIR)
val_images_normalized = process_images_in_directory(VAL_IMAGES_DIR)

print(f"Nombre d'images normalisées dans le dossier d'entraînement : {len(train_images_normalized)}")
print(f"Nombre d'images normalisées dans le dossier de validation : {len(val_images_normalized)}")


In [ ]:
# Create a data.yaml file required by yolo
class_names = train['class'].unique().tolist()  # Récupère la liste des noms de classes uniques dans le jeu de données d'entraînement
num_classes = len(class_names)                  # Calcule le nombre de classes

data_yaml = {
    # getcwd() est une fonction qui retourne une chaîne de caractères représentant le chemin absolu du répertoire de travail actuel
    'train': os.getcwd()+'/' + str(TRAIN_IMAGES_DIR),    # Définit le chemin du répertoire des images d'entraînement
    'val': os.getcwd()+'/' + str(VAL_IMAGES_DIR),        # Définit le chemin du répertoire des images de validation
    'test': os.getcwd()+'/' + str(TEST_IMAGES_DIR),      # Définit le chemin du répertoire des images de test
    'nc': num_classes,                                   # Définit le nombre de classes
    'names': class_names                                 # Définit la liste des noms de classes
}


yaml_path = 'data.yaml'                            # Définit le chemin du fichier YAML à créer

with open(yaml_path, 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)            # Écrit le dictionnaire data_yaml dans le fichier YAML

# Preview data yaml file
data_yaml                                             # Affiche le contenu du fichier YAML créé

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm

# Fonction pour charger les annotations YOLO à partir d'un fichier
def load_annotations(label_path):
    with open(label_path, 'r') as f:
        lines = f.readlines()
    boxes = []
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        boxes.append((class_id, x_center, y_center, width, height))
    return boxes

# Fonction pour afficher une image avec ses boîtes englobantes
def plot_image_with_boxes(image_path, boxes):
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    h, w, _ = image.shape
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    
    for box in boxes:
        class_id, x_center, y_center, width, height = box
        xmin = int((x_center - width / 2) * w)
        ymin = int((y_center - height / 2) * h)
        xmax = int((x_center + width / 2) * w)
        ymax = int((y_center + height / 2) * h)
        
        plt.gca().add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                          edgecolor='red', facecolor='none', linewidth=2))
        plt.text(xmin, ymin - 10, f'Class {int(class_id)}', color='red', fontsize=12, weight='bold')
    
    plt.axis('off')
    plt.show()

# Chemins de base pour les images et labels
BASE_DIR = Path("dataset")
IMAGE_DIR = IMAGES_DIR
LABEL_DIR = LABELS_DIR

# Extensions d'image possibles
extensions = ['.jpg', '.png']

# Parcourir les sous-dossiers (train, val, test)
for split in ['train', 'val', 'test']:
    image_split_dir = IMAGE_DIR / split
    label_split_dir = LABEL_DIR / split
    
    # Parcourir les classes (class_0, class_1)
    for class_folder in label_split_dir.iterdir():
        if class_folder.is_dir():
            class_id = int(class_folder.name.split('_')[-1])
            
            # Parcourir chaque fichier d'annotation dans le dossier de classe
            for label_file in tqdm(list(class_folder.glob("*.txt"))[:3], desc=f"Plotting images for {split}/{class_folder.name}"):
                image_name = label_file.stem  # Nom sans extension
                image_path = None
                
                # Recherchez l'image avec l'une des extensions possibles dans le dossier images
                for ext in extensions:
                    potential_path = image_split_dir / class_folder.name / f"{image_name}{ext}"
                    if potential_path.exists():
                        image_path = potential_path
                        break
                
                if image_path:
                    boxes = load_annotations(label_file)
                    print(f"Plotting {image_path.name} with {len(boxes)} bounding boxes.")
                    plot_image_with_boxes(image_path, boxes)
                else:
                    print(f"No image found for annotation {label_file.name} in {split}/{class_folder.name}.")


In [ ]:
# Récupération sécurisée de la clé API de Weights & Biases (WandB)

user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("yolo_key")

# Définition de la clé API dans les variables d'environnement pour WandB
os.environ['WANDB_API_KEY'] = wandb_api_key

In [ ]:


# train_directory = TRAIN_IMAGES_DIR

# from skopt import BayesSearchCV
# from skopt.space import Real, Integer, Categorical
# from sklearn.base import BaseEstimator
# from ultralytics import YOLO
# import os
# import numpy as np

# class YOLOEstimator(BaseEstimator):
#     def __init__(self, model_path='/kaggle/input/yolo_fine_8l/tensorflow2/default/1/best.pt', data_yaml='data.yaml'):
#         self.model_path = model_path
#         self.data_yaml = data_yaml
#         self.model = None

#     def fit(self, X, y=None, **params):
#         self.model = YOLO(self.model_path)
        
#         # Entraînement avec les hyperparamètres fournis
#         train_args = {

  
#             'data': self.data_yaml,
#             'epochs': int(params.get('epochs', 25)),
#             'imgsz': int(params.get('imgsz', 640)),
#             'batch': int(params.get('batch', 32)),
#             'device': params.get('device', [0, 1]),
#             'lr0': float(params.get('lr0', 0.0017399892628386226)),
#             'lrf': float(params.get('lrf', 0.01)),
#             'momentum': float(params.get('momentum', 0.8369417969048814)),
#             'weight_decay': float(params.get('weight_decay', 0.0005607011624693186)),
#         }
        
        
#         try:
#             results = self.model.train(**train_args)
#             self.best_map = results.maps.get('mAP50-95(B)', 0)
#         except Exception as e:
#             print(f"Erreur lors de l'entraînement : {e}")
#             self.best_map = 0
        
#         return self

#     def score(self, X, y=None):
#         if self.model is None:
#             return 0
#         try:
#             results = self.model.val()
#             return results.maps.get('mAP50-95(B)', 0)
#         except Exception as e:
#             print(f"Erreur lors de la validation : {e}")
#             return 0

#     def get_params(self, deep=True):
#         return {
#             'model_path': self.model_path,
#             'data_yaml': self.data_yaml
#         }

#     def set_params(self, **params):
#         for key, value in params.items():
#             setattr(self, key, value)
#         return self

# # Définition de l'espace de recherche pour chaque hyperparamètre à optimiser
# search_space = {
#     'epochs': Integer(5, 50),  # Nombre d'époques ajusté
#     'imgsz': Integer(400, 1024),  # Taille des images d'entrée
#     'batch': Integer(16, 32),  # Taille du lot ajustée
#     'lr0': Real(1e-4, 1e-2),          # Taux d'apprentissage initial
#     'weight_decay': Real(1e-5, 1e-3), # Décroissance des poids
#     'momentum': Real(0.7, 0.95),       # Momentum
#     'lrf':  Real(1e-4, 1e-2)      
# }



# # Nombre total d'itérations pour la recherche bayesienne
# n_iterations = 10


# # Récupérer les chemins d'images du dossier d'entraînement
# X_train = [os.path.join(train_directory, img) for img in os.listdir(train_directory) if img.endswith(('.jpg', '.jpeg', '.png'))]
# y_train = None  # Pas besoin d'utiliser y_train ici car nous utilisons les chemins directement

# # Entraînement avec YOLOv9 et recherche bayesienne
# optimizer = BayesSearchCV(
#     estimator=YOLOEstimator(model_path='/kaggle/input/yolo_fine_8l/tensorflow2/default/1/best.pt', data_yaml='data.yaml'),
#     search_spaces=search_space,
#     n_iter=n_iterations,  # Nombre d'itérations pour la recherche bayesienne
#     cv=3,  # Utiliser une validation croisée
#     n_jobs=-1,  # Utilisation de tous les processeurs disponibles
#     verbose=False
# )

# # Ajustement du modèle avec le jeu de données d'entraînement
# optimizer.fit(X_train, y_train)

# # Meilleurs hyperparamètres trouvés
# best_hyperparams = optimizer.best_params_
# print("Meilleurs hyperparamètres trouvés : ", best_hyperparams)





# # def objective(trial):
# #     # Définir les hyperparamètres à optimiser avec Optuna
# #     train_args = {
# #             'data': 'data.yaml',
# #             'epochs': 5,
# #             'patience': 10,
# #             'batch': 16,
# #             'imgsz': 320,
# #             'save': True,
# #             'save_period': -1,
# #             'cache': False,
# #             'device': [0,1],  
# #             'workers': 8,
# #             'project': "optuna_trials",
# #             'name': f"trial_{trial.number}",
# #             'optimizer': trial.suggest_categorical('optimizer', ['SGD', 'Adam', 'AdamW']),
# #             'cos_lr': trial.suggest_categorical('cos_lr', [True, False]),
# #             'rect': trial.suggest_categorical('rect', [True, False]),
# #             'amp': trial.suggest_categorical('amp', [True, False]),
# #             'multi_scale': trial.suggest_categorical('multi_scale', [True, False]),
# #             'lr0': trial.suggest_loguniform('lr0', 1e-4, 1e-2),
# #             'lrf': trial.suggest_uniform('lrf', 0.01, 0.3),
# #             'momentum': trial.suggest_uniform('momentum', 0.85, 0.99),
# #             'weight_decay': trial.suggest_loguniform('weight_decay', 1e-5, 1e-3),
# #             'warmup_epochs': trial.suggest_uniform('warmup_epochs', 0, 5),
# #             'warmup_momentum': trial.suggest_uniform('warmup_momentum', 0.7, 0.9),
# #             'warmup_bias_lr': trial.suggest_uniform('warmup_bias_lr', 0.05, 0.2),
# #             'box': trial.suggest_uniform('box', 0.02, 0.2),
# #             'cls': trial.suggest_uniform('cls', 0.2, 4.0),
# #             'dfl': trial.suggest_uniform('dfl', 0.5, 2.0),
# #             'pose': trial.suggest_uniform('pose', 1.0, 20.0),
# #             'kobj': trial.suggest_uniform('kobj', 0.1, 2.0),
# #             'label_smoothing': trial.suggest_uniform('label_smoothing', 0.0, 0.1),
# #             'hsv_h': trial.suggest_uniform('hsv_h', 0.0, 0.1),
# #             'hsv_s': trial.suggest_uniform('hsv_s', 0.5, 1.0),
# #             'hsv_v': trial.suggest_uniform('hsv_v', 0.2, 1.0),
# #             'degrees': trial.suggest_uniform('degrees', 0, 5),
# #             'translate': trial.suggest_uniform('translate', 0, 0.5),
# #             'scale': trial.suggest_uniform('scale', 0.0, 1.0),
# #             'shear': trial.suggest_uniform('shear', 0.0, 5.0),
# #             'perspective': trial.suggest_uniform('perspective', 0.0, 0.001),
# #             'flipud': trial.suggest_uniform('flipud', 0.0, 0.5),
# #             'fliplr': trial.suggest_uniform('fliplr', 0.0, 0.5),
# #             'mosaic': trial.suggest_uniform('mosaic', 0.0, 1.0),
# #             'mixup': trial.suggest_uniform('mixup', 0.0, 1.0),
# #             'copy_paste': trial.suggest_uniform('copy_paste', 0.0, 0.5)
# #         }

# #         # Configurer et entraîner le modèle YOLO
# #     model = YOLO("/kaggle/input/yolo_fine_8l/tensorflow2/default/1/best.pt")  # Assurez-vous que le fichier modèle est compatible
# #     results = model.train(**train_args)
# #     result = model.val()
# #         # Définir la métrique à optimiser (ex: mAP@0.5)
# #     map_50 = result.box.map50  # mAP@0.5
    
# #     return map_50

# #     # Lancer l'optimisation avec Optuna
# # study = optuna.create_study(direction="maximize")
# # study.optimize(objective, n_trials=10)

# #     # Meilleure configuration d'hyperparamètres
# # print("Best hyperparameters:", study.best_params)


In [ ]:
# import random
# from deap import base, creator, tools, algorithms
# from ultralytics import YOLO

# # Fonction pour évaluer le modèle avec les hyperparamètres
# def evaluate(individual):
#     # Extraire les valeurs des individus
#     (optimizer, cos_lr, rect, amp, multi_scale, lr0, lrf, momentum, weight_decay, warmup_epochs,
#      warmup_momentum, warmup_bias_lr, box, cls, dfl, pose, kobj, label_smoothing,
#      hsv_h, hsv_s, hsv_v, degrees, translate, scale, shear, perspective, flipud,
#      fliplr, mosaic, mixup, copy_paste) = individual

#     # Configurer YOLO avec les hyperparamètres
#     model = YOLO("yolov8l.pt")
#     train_args = {
#         'data': 'data.yaml',
#         'epochs': 10,
#         'patience': 5,
#         'batch': 16,
#         'imgsz': 340,
#         'save': True,
#         'optimizer': optimizer,
#         'cos_lr': cos_lr,
#         'rect': rect,
#         'amp': amp,
#         'multi_scale': multi_scale,
#         'lr0': lr0,
#         'lrf': lrf,
#         'momentum': momentum,
#         'weight_decay': weight_decay,
#         'warmup_epochs': warmup_epochs,
#         'warmup_momentum': warmup_momentum,
#         'warmup_bias_lr': warmup_bias_lr,
#         'box': box,
#         'cls': cls,
#         'dfl': dfl,
#         'pose': pose,
#         'kobj': kobj,
#         'label_smoothing': label_smoothing,
#         'hsv_h': hsv_h,
#         'hsv_s': hsv_s,
#         'hsv_v': hsv_v,
#         'degrees': degrees,
#         'translate': translate,
#         'scale': scale,
#         'shear': shear,
#         'perspective': perspective,
#         'flipud': flipud,
#         'fliplr': fliplr,
#         'mosaic': mosaic,
#         'mixup': mixup,
#         'copy_paste': copy_paste,
#         'device': [0,1],
#     }

#     # Entraîner le modèle
#     results = model.train(**train_args)
    
#     result = model.val()
#     # Définir la métrique à optimiser (ex: mAP@0.5)
#     map_50 = result.box.map50  # mAP@0.5
#     return map_50,

# # Supprimer les classes existantes
# if "FitnessMax" in creator.__dict__:
#     del creator.FitnessMax
# if "Individual" in creator.__dict__:
#     del creator.Individual

# # Initialisation de DEAP
# creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# creator.create("Individual", list, fitness=creator.FitnessMax)

# toolbox = base.Toolbox()
# toolbox.register("optimizer", random.choice, ["SGD", "Adam", "AdamW"])
# toolbox.register("cos_lr", random.choice, [True, False])
# toolbox.register("rect", random.choice, [True, False])
# toolbox.register("amp", random.choice, [True, False])
# toolbox.register("multi_scale", random.choice, [True, False])
# toolbox.register("lr0", random.uniform, 1e-4, 1e-2)
# toolbox.register("lrf", random.uniform, 0.0, 1.0)
# toolbox.register("momentum", random.uniform, 0.0, 1.0)
# toolbox.register("weight_decay", random.uniform, 1e-5, 1e-3)
# toolbox.register("warmup_epochs", random.uniform, 0, 5)
# toolbox.register("warmup_momentum", random.uniform, 0.0, 1.0)
# toolbox.register("warmup_bias_lr", random.uniform, 0.0, 1.0)
# toolbox.register("box", random.uniform, 0.0, 1.0)
# toolbox.register("cls", random.uniform, 0.0, 1.0)
# toolbox.register("dfl", random.uniform, 0.0, 1.0)
# toolbox.register("pose", random.uniform, 1.0, 20.0)
# toolbox.register("kobj", random.uniform, 0.0, 1.0)
# toolbox.register("label_smoothing", random.uniform, 0.0, 0.1)
# toolbox.register("hsv_h", random.uniform, 0.0 , 1.0)
# toolbox.register("hsv_s", random.uniform, 0.0, 1.0)
# toolbox.register("hsv_v", random.uniform, 0.0, 1.0)
# toolbox.register("degrees", random.uniform, -180, 180)
# toolbox.register("translate", random.uniform, 0.0, 1.0)
# toolbox.register("scale", random.uniform, 0.0, 1.0)
# toolbox.register("shear", random.uniform, -180, 180)
# toolbox.register("perspective", random.uniform, 0.0 , 1.0)
# toolbox.register("flipud", random.choice, [True, False])
# toolbox.register("fliplr", random.choice, [True, False])
# toolbox.register("mosaic", random.uniform, 0.0, 1.0)
# toolbox.register("mixup", random.uniform, 0.0, 1.0)
# toolbox.register("copy_paste", random.uniform, 0.0, 1.0)

# toolbox.register("individual", tools.initCycle, creator.Individual,
#                  (toolbox.optimizer, toolbox.cos_lr, toolbox.rect, toolbox.amp, toolbox.multi_scale,
#                   toolbox.lr0, toolbox.lrf, toolbox.momentum, toolbox.weight_decay, toolbox.warmup_epochs,
#                   toolbox.warmup_momentum, toolbox.warmup_bias_lr, toolbox.box, toolbox.cls, toolbox.dfl,
#                   toolbox.pose, toolbox.kobj, toolbox.label_smoothing, toolbox.hsv_h, toolbox.hsv_s,
#                   toolbox.hsv_v, toolbox.degrees, toolbox.translate, toolbox.scale, toolbox.shear,
#                   toolbox.perspective, toolbox.flipud, toolbox.fliplr, toolbox.mosaic, toolbox.mixup, toolbox.copy_paste), n=1)

# toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# toolbox.register("evaluate", evaluate)

# def custom_crossover(ind1, ind2):
#     for i in range(5, len(ind1)):  # Les indices 0 à 4 sont booléens
#         if random.random() < 0.5:
#             ind1[i], ind2[i] = ind2[i], ind1[i]
#     return ind1, ind2

# toolbox.register("mate", custom_crossover)

# def custom_mutate(individual):
#     for i in range(5, len(individual)):
#         if random.random() < 0.2:
#             individual[i] += random.gauss(0, 0.1)
#             individual[i] = min(max(individual[i], 0.0), 1.0)
#     return individual,

# toolbox.register("mutate", custom_mutate)
# toolbox.register("select", tools.selTournament, tournsize=3)

# def genetic_algorithm():
#     population = toolbox.population(n=10)
#     ngen = 5
#     cxpb, mutpb = 0.5, 0.2
#     for gen in range(ngen):
#         offspring = algorithms.varAnd(population, toolbox, cxpb, mutpb)
#         fits = map(toolbox.evaluate, offspring)
#         for fit, ind in zip(fits, offspring):
#             ind.fitness.values = fit
#         population = toolbox.select(offspring, k=len(population))
#     best_ind = tools.selBest(population, 1)[0]
#     print("Best hyperparameters:", best_ind)

# genetic_algorithm()


In [ ]:

# Chargement des modèles YOLO pré-entraînés

model = YOLO("yolov8l.pt")  # charge un modèle YOLO pré-entrainé a partir du fichier

In [ ]:
# Afficher des informations sur le modèle YOLOv9
model.info()

In [ ]:
# # Hyperparamètres avec les valeurs respectives
# parameters = {
#     'data': 'data.yaml',
#     'epochs': 10,
#     'patience': 5,
#     'batch': 16,
#     'imgsz': 640,
#     'optimizer': 'SGD',
#     'cos_lr': True,
#     'rect': False,
#     'amp': True,
#     'multi_scale': False,
#     'lr0': 0.005551929282120419,
#     'lrf': 0.4579592312901949,
#     'momentum': 0.44236721090471254,
#     'weight_decay': 0.0009518886980375375,
#     'warmup_epochs': 3.9408752317959497,
#     'warmup_momentum': 0.09402580543150785,
#     'warmup_bias_lr': 0.9768579038881372,
#     'box': 0.9612251516116247,
#     'cls': 0.5467338228386568,
#     'dfl': 0.5583251507678315,
#     'pose': 6.696711378447303,
#     'kobj': 0.16850162671180136,
#     'label_smoothing': 0.04103172349468214,
#     'hsv_h': 0.9455029840216489,
#     'hsv_s': 0.222983704358233,
#     'hsv_v': 0.19951589617132215,
#     'degrees': -115.19784272109167,
#     'translate': 0.0244370204819222,
#     'scale': 0.12042183595275147,
#     'shear': -48.34816491677475,
#     'perspective': 0.12116574738152142,
#     'flipud': True,
#     'fliplr': True,
#     'mosaic': 0.4704951902504716,
#     'mixup': 0.5949810652131459,
#     'copy_paste': 0.04963149347304907,
#     'device': [0, 1] 
# }

# model.train(**parameters)

In [ ]:
# # # Lister tous les chemins des images dans le répertoire d'entraînement
# # image_paths = list(train_directory.rglob('*.[jJpP][pPnN][gG]'))[:5]  # Limiter à 5 images


# # Fonction pour charger une image
# def load_image_as_tensor(image_path):
#     # Charger l'image avec OpenCV
#     image = cv2.imread(image_path)
    
#     # Convertir l'image de BGR (par défaut dans OpenCV) à RGB
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
#     # Appliquer les augmentations
#     augmented = transform(image=image)
#     augmented_image = augmented["image"]
    
#     return augmented_image
# # Fonction pour afficher les images
# def display_images(image_paths):
#     plt.figure(figsize=(640, 640))  # Ajuster la taille de la figure
#     for i, img_path in enumerate(image_paths):
#         image = cv2.imread(str(img_path))
#         if image is None:
#             print(f"Erreur de chargement de l'image : {img_path}")
#             continue  # Passer à la prochaine image si celle-ci n'a pas pu être chargée
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convertir de BGR à RGB
        
#         plt.subplot(1, 5, i + 1)  # Créer une grille de 1 ligne et 5 colonnes
#         plt.imshow(image)
#         plt.axis('off')  # Masquer les axes
#         plt.title(img_path.stem)  # Afficher le nom de l'image sans extension

#     plt.tight_layout()
#     plt.show()

# # Afficher les cinq premières images
# #display_images('../input/images/id_fe3rxq4495_aug0.jpg')

In [ ]:
model.train(
        data         = 'data.yaml',         # Chemin vers les annotations yaml
        epochs       = 22,              # Nombre d'époques
        imgsz        = 416,              # Taille des images
        batch        = 24,               # Taille du lot
        device       = [0,1],               # Utilisation du GPU
        lr0          = 0.004282751034955861,
        lrf          = 0.00274431132496402,
        momentum     = 0.9409580968621439,
        weight_decay = 0.0008902799350696401
    )


In [ ]:

# # Validate the model on the validation set
# model = YOLO('/' + os.getcwd() + '/runs/detect/train/weights/best.pt')  # Charge le modèle YOLO à partir du fichier 'best.pt' qui contient les poids du meilleur modèle entraîné
# results = model.val()      # Évalue le modèle sur le jeu de validation et stocke les résultats de la validation


# Vérifier la disponibilité des GPUs
if torch.cuda.is_available() and torch.cuda.device_count() > 1:
    print(f"Nombre de GPUs disponibles : {torch.cuda.device_count()}")
    device = [0, 1]  # Utilise les deux premiers GPUs
    print(f"Utilisation des GPUs : {device}")
else:
    print("Pas assez de GPUs disponibles, utilisation du CPU ou d'un seul GPU.")
    device = [0] if torch.cuda.is_available() else 'cpu'

# Charger le modèle
model_path = '/' + os.getcwd() + '/runs/detect/train/weights/best.pt'
if os.path.exists(model_path):
    model = YOLO(model_path)
else:
    raise FileNotFoundError(f"Le fichier de modèle {model_path} n'existe pas.")

# Valider le modèle sur l'ensemble de validation
try:
    results = model.val(
        device=device
    )
    
    # Afficher les résultats
    
    # Vous pouvez accéder à des métriques spécifiques comme ceci :
    
    print(f"Precision: {results.box.p}")
    print(f"Recall: {results.box.r}")
    # Extraire mAP@0.5 et mAP@[0.5:0.95]
    map_50 = results.box.map50  # mAP@0.5
    map_50_95 = results.box.map  # mAP@[0.5:0.95]

    print(f"mAP@0.5 (VOC): {map_50:.4f}")
    print(f"mAP@[0.5:0.95] (COCO): {map_50_95:.4f}")
        
except Exception as e:
    print(f"Une erreur s'est produite lors de la validation : {e}")

In [ ]:
# Load the trained YOLO model  train
model = YOLO('/'+os.getcwd()+'/runs/detect/train/weights/best.pt')

# Path to the test images directory
test_dir_path = '/'+os.getcwd()+'/datasets/dataset/images/test'

# Get a list of all image files in the test directory
image_files = os.listdir(test_dir_path)  # Récupère la liste de tous les fichiers image dans le répertoire de test

# Initialize an empty list to store the results for all images
all_data = []  # Initialise une liste vide pour stocker les résultats de toutes les images

# Iterate through each image in the directory
for image_file in tqdm(image_files):  # Parcourt chaque fichier image dans le répertoire, avec une barre de progression
    # Full path to the image
    img_path = os.path.join(test_dir_path, image_file)  # Crée le chemin complet vers l'image

    # Make predictions on the image
    results = model(img_path)  # Effectue des predictions sur l'image à l'aide du modèle

    # Extract bounding boxes, confidence scores, and class labels
    boxes = results[0].boxes.xyxy.tolist()  # Récupère les boîtes englobantes au format xyxy
    classes = results[0].boxes.cls.tolist()  # Récupère les indices de classe
    confidences = results[0].boxes.conf.tolist()  # Récupère les scores de confiance
    names = results[0].names  # Récupère le dictionnaire des noms de classe

    if not boxes:  # Vérifie s'il n'y a pas de détections
        # If no detections, add NEG as the class
        all_data.append({
            'Image_ID': image_file,  # ID de l'image
            'class': 'NEG',           # Classe par défaut 'NEG' (négatif)
            'confidence': 1.0,        # Valeur par défaut (vous pouvez la définir comme valeur standard)
            'ymin': 0,                # Valeur par défaut (pas de détection)
            'xmin': 0,                # Valeur par défaut (pas de détection)
            'ymax': 0,                # Valeur par défaut (pas de détection)
            'xmax': 0                 # Valeur par défaut (pas de détection)
        })
    else:
        # Iterate through the results for this image
        for box, cls, conf in zip(boxes, classes, confidences):  # Parcourt les résultats de cette image
            x1, y1, x2, y2 = box  # Récupère les coordonnées de la boîte englobante
            detected_class = names[int(cls)]  # Récupère le nom de la classe à partir du dictionnaire de noms

            # Add the result to the all_data list
            all_data.append({
                'Image_ID': image_file,  # ID de l'image
                'class': detected_class,  # Classe détectée
                'confidence': conf,       # Score de confiance
                'ymin': y1,               # Coordonnée y minimale
                'xmin': x1,               # Coordonnée x minimale
                'ymax': y2,               # Coordonnée y maximale
                'xmax': x2                # Coordonnée x maximale
            })
# Convert the list to a DataFrame for all images
sub = pd.DataFrame(all_data)

In [ ]:
sub.head()  # Affiche les premières lignes du DataFrame 'sub' pour donner un aperçu de son contenu

In [ ]:
sub['class'].value_counts()  # Affiche le nombre d'occurrences de chaque classe dans la colonne 'class' du DataFrame 'sub'

In [ ]:
# Create submission file to be uploaded to Zindi for scoring
sub.to_csv('benchmark_submission.csv', index = False)

In [ ]:
import shutil

# Définir le chemin du fichier d'origine et le chemin de destination
model_path = '/' + os.getcwd() + '/runs/detect/train/weights/best.pt'
destination_path = os.getcwd()  # Chemin de destination pour la copie

# Copier le fichier
shutil.copy(model_path, destination_path)
print(f"Le fichier a été copié dans : {destination_path}")

# import pandas as pd

# import numpy as np

# # Fonction pour calculer l'AP à 11 points de rappel (VOC 2007)
# def voc2007_ap(recalls, precisions):
#     ap = 0.0
#     for t in np.linspace(0, 1, 11):  # 11 points de rappel
#         if np.sum(recalls >= t) == 0:
#             p = 0
#         else:
#             p = np.max(precisions[recalls >= t])
#         ap += p / 11  # moyenne des précisions aux 11 points
#     return ap

# # Exemple de fonction pour calculer les métriques mAP pour chaque classe
# def voc2007_mAP(predictions, ground_truths, iou_threshold=0.5):
#     aps = []
#     for class_id in np.unique([pred['class_id'] for pred in predictions]):
#         # Calcul des TP, FP, et du rappel pour la classe en question
#         recalls, precisions = compute_precision_recall(predictions, ground_truths, class_id, iou_threshold)
#         ap = voc2007_ap(recalls, precisions)
#         aps.append(ap)
#     return np.mean(aps)  # mAP

# def voc2012_ap(recalls, precisions):
#     # Interpolation pour avoir une courbe de précision décroissante
#     precisions = np.maximum.accumulate(precisions[::-1])[::-1]
#     # Calcul de l'aire sous la courbe (AUC) pour obtenir l'AP
#     ap = np.trapz(precisions, recalls)
#     return ap

# def voc2012_mAP(predictions, ground_truths, iou_threshold=0.5):
#     aps = []
#     for class_id in np.unique([pred['class_id'] for pred in predictions]):
#         recalls, precisions = compute_precision_recall(predictions, ground_truths, class_id, iou_threshold)
#         ap = voc2012_ap(recalls, precisions)
#         aps.append(ap)
#     return np.mean(aps)

# def coco_ap(recalls, precisions):
#     precisions = np.maximum.accumulate(precisions[::-1])[::-1]
#     ap = np.trapz(precisions, recalls)
#     return ap

# def coco_mAP(predictions, ground_truths):
#     aps = []
#     for iou_threshold in np.arange(0.5, 1.0, 0.05):  # Seuils de IoU de 0.50 à 0.95
#         aps_per_class = []
#         for class_id in np.unique([pred['class_id'] for pred in predictions]):
#             recalls, precisions = compute_precision_recall(predictions, ground_truths, class_id, iou_threshold)
#             ap = coco_ap(recalls, precisions)
#             aps_per_class.append(ap)
#         aps.append(np.mean(aps_per_class))
#     return np.mean(aps)  # mAP


# def compute_precision_recall(predictions, ground_truths, class_id, iou_threshold):
#     # Extraire les prédictions et vérités de terrain pour la classe donnée
#     preds = [p for p in predictions if p['class_id'] == class_id]
#     gts = [gt for gt in ground_truths if gt['class_id'] == class_id]

#     # Trier les prédictions par score de confiance décroissant
#     preds = sorted(preds, key=lambda x: x['confidence'], reverse=True)

#     tp = np.zeros(len(preds))
#     fp = np.zeros(len(preds))
#     detected = set()  # Pour suivre les objets déjà détectés

#     # Calcul des TP et FP
#     for i, pred in enumerate(preds):
#         best_iou = 0
#         best_gt = -1
#         for j, gt in enumerate(gts):
#             if j in detected:
#                 continue  # Passer les objets déjà détectés

#             iou = compute_iou(pred['bbox'], gt['bbox'])
#             if iou > best_iou:
#                 best_iou = iou
#                 best_gt = j

#         if best_iou >= iou_threshold:
#             tp[i] = 1
#             detected.add(best_gt)
#         else:
#             fp[i] = 1

#     # Calcul des rappels et précisions cumulés
#     tp_cumsum = np.cumsum(tp)
#     fp_cumsum = np.cumsum(fp)
#     recalls = tp_cumsum / len(gts)
#     precisions = tp_cumsum / (tp_cumsum + fp_cumsum)

#     return recalls, precisions

# def compute_iou(boxA, boxB):
#     xA = max(boxA[0], boxB[0])
#     yA = max(boxA[1], boxB[1])
#     xB = min(boxA[2], boxB[2])
#     yB = min(boxA[3], boxB[3])

#     interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
#     boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
#     boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

#     iou = interArea / float(boxAArea + boxBArea - interArea)
#     return iou


# # Charger le fichier de soumission
# submission_df = pd.read_csv('benchmark_submission.csv')
# real_df = pd.read_csv('../input/Train.csv')

# # Structure les prédictions
# predictions = []
# for _, row in submission_df.iterrows():
#     bbox = [row['xmin'], row['ymin'], row['xmax'], row['ymax']]
#     prediction = {
#         'Image_ID': row['Image_ID'],
#         'class_id': row['class'],
#         'confidence': row['confidence'],
#         'bbox': bbox
#     }
#     predictions.append(prediction)


# ground_truths = []

# for _, row in real_df.iterrows():
#     bbox = [row['xmin'], row['ymin'], row['xmax'], row['ymax']]
#     ground_truth = {
#         'Image_ID': row['Image_ID'],
#         'class_id': row['class'],
#         'confidence': row['confidence'],
#         'bbox': bbox
#     }
#     ground_truths.append(ground_truth)


# # Fonction pour calculer mAP selon VOC 2007
# voc2007_mAP_score = voc2007_mAP(predictions, ground_truths)

# # Fonction pour calculer mAP selon VOC 2012
# voc2012_mAP_score = voc2012_mAP(predictions, ground_truths)

# # Fonction pour calculer mAP selon COCO
# coco_mAP_score = coco_mAP(predictions, ground_truths)

# # Affichage des résultats
# print("mAP selon VOC 2007:", voc2007_mAP_score)
# print("mAP selon VOC 2012:", voc2012_mAP_score)
# print("mAP selon COCO:", coco_mAP_score)


In [ ]:

# train_directory = TRAIN_IMAGES_DIR
# from skopt import BayesSearchCV
# from skopt.space import Real, Integer, Categorical
# from sklearn.base import BaseEstimator
# from ultralytics import YOLO
# import os
# import numpy as np

# class YOLOEstimator(BaseEstimator):
#     def __init__(self, model_path='yolov9c.pt', data_yaml='data.yaml'):
#         self.model_path = model_path
#         self.data_yaml = data_yaml
#         self.model = None

#     def fit(self, X, y=None, **params):
#         self.model = YOLO(self.model_path)
        
#         # Entraînement avec les hyperparamètres fournis
#         train_args = {
#             'data': self.data_yaml,
#             'epochs': int(params.get('epochs', 20)),
#             'imgsz': int(params.get('imgsz', 640)),
#             'batch': int(params.get('batch', 32)),
#             'device': params.get('device', [0, 1]),
#             'lr0': float(params.get('lr0', 0.01)),
#             'lrf': float(params.get('lrf', 0.01)),
#             'momentum': float(params.get('momentum', 0.937)),
#             'weight_decay': float(params.get('weight_decay', 0.0005)),
#             'warmup_epochs': int(params.get('warmup_epochs', 3)),
#             'warmup_momentum': float(params.get('warmup_momentum', 0.8)),
#             'warmup_bias_lr': float(params.get('warmup_bias_lr', 0.1)),
#             'hsv_h': float(params.get('hsv_h', 0.015)),
#             'hsv_s': float(params.get('hsv_s', 0.7)),
#             'hsv_v': float(params.get('hsv_v', 0.4)),
#             'mosaic': float(params.get('mosaic', 1.0)),
#             'mixup': float(params.get('mixup', 0.5)),
#             'cache': params.get('cache', True),
#             'val': params.get('val', True),
#             'amp': True,
#             'verbose': False
#         }
        
        
#         try:
#             results = self.model.train(**train_args)
#             self.best_map = results.maps.get('mAP50-95(B)', 0)
#         except Exception as e:
#             print(f"Erreur lors de l'entraînement : {e}")
#             self.best_map = 0
        
#         return self

#     def score(self, X, y=None):
#         if self.model is None:
#             return 0
#         try:
#             results = self.model.val()
#             return results.maps.get('mAP50-95(B)', 0)
#         except Exception as e:
#             print(f"Erreur lors de la validation : {e}")
#             return 0

#     def get_params(self, deep=True):
#         return {
#             'model_path': self.model_path,
#             'data_yaml': self.data_yaml
#         }

#     def set_params(self, **params):
#         for key, value in params.items():
#             setattr(self, key, value)
#         return self

# # Définition de l'espace de recherche pour chaque hyperparamètre à optimiser
# search_space = {
#     'epochs': Integer(5, 50),  # Nombre d'époques ajusté
#     'imgsz': Integer(640, 1024),  # Taille des images d'entrée
#     'batch': Integer(16, 32),  # Taille du lot ajustée
#     'lr0': Real(1e-4, 1e-2),          # Taux d'apprentissage initial
#     'weight_decay': Real(1e-5, 1e-3), # Décroissance des poids
#     'momentum': Real(0.7, 0.95)       # Momentum

# }



# # Nombre total d'itérations pour la recherche bayesienne
# n_iterations = 10


# # Récupérer les chemins d'images du dossier d'entraînement
# X_train = [os.path.join(train_directory, img) for img in os.listdir(train_directory) if img.endswith(('.jpg', '.jpeg', '.png'))]
# y_train = None  # Pas besoin d'utiliser y_train ici car nous utilisons les chemins directement

# # Entraînement avec YOLOv9 et recherche bayesienne
# optimizer = BayesSearchCV(
#     estimator=YOLOEstimator(model_path='yolov9c.pt', data_yaml='data.yaml'),
#     search_spaces=search_space,
#     n_iter=n_iterations,  # Nombre d'itérations pour la recherche bayesienne
#     cv=3,  # Utiliser une validation croisée
#     n_jobs=-1,  # Utilisation de tous les processeurs disponibles
#     verbose=False
# )

# # Ajustement du modèle avec le jeu de données d'entraînement
# optimizer.fit(X_train, y_train)

# # Meilleurs hyperparamètres trouvés
# best_hyperparams = optimizer.best_params_
# print("Meilleurs hyperparamètres trouvés : ", best_hyperparams)

#--------------------------------------------------------------------------------------------------
# import random
# from deap import base, creator, tools, algorithms
# from ultralytics import YOLO

# # Fonction pour évaluer le modèle avec les hyperparamètres
# def evaluate(individual):
#     # Extraire les valeurs des individus
#     (optimizer, cos_lr, rect, amp, multi_scale, lr0, lrf, momentum, weight_decay, warmup_epochs,
#      warmup_momentum, warmup_bias_lr, box, cls, dfl, pose, kobj, label_smoothing,
#      hsv_h, hsv_s, hsv_v, degrees, translate, scale, shear, perspective, flipud,
#      fliplr, mosaic, mixup, copy_paste) = individual

#     # Configurer YOLO avec les hyperparamètres
#     model = YOLO("yolov9c.pt")
#     train_args = {
#         'data': 'data.yaml',
#         'epochs': 100,
#         'patience': 100,
#         'batch': 16,
#         'imgsz': 640,
#         'save': True,
#         'optimizer': optimizer,
#         'cos_lr': cos_lr,
#         'rect': rect,
#         'amp': amp,
#         'multi_scale': multi_scale,
#         'lr0': lr0,
#         'lrf': lrf,
#         'momentum': momentum,
#         'weight_decay': weight_decay,
#         'warmup_epochs': warmup_epochs,
#         'warmup_momentum': warmup_momentum,
#         'warmup_bias_lr': warmup_bias_lr,
#         'box': box,
#         'cls': cls,
#         'dfl': dfl,
#         'pose': pose,
#         'kobj': kobj,
#         'label_smoothing': label_smoothing,
#         'hsv_h': hsv_h,
#         'hsv_s': hsv_s,
#         'hsv_v': hsv_v,
#         'degrees': degrees,
#         'translate': translate,
#         'scale': scale,
#         'shear': shear,
#         'perspective': perspective,
#         'flipud': flipud,
#         'fliplr': fliplr,
#         'mosaic': mosaic,
#         'mixup': mixup,
#         'copy_paste': copy_paste
#     }

#     # Entraîner le modèle
#     results = model.train(**train_args)
#     metrics = results.metrics

#     # Retourner la métrique (mAP) pour maximiser
#     return metrics['map_0.5'],

# # Initialisation de DEAP
# creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# creator.create("Individual", list, fitness=creator.FitnessMax)

# toolbox = base.Toolbox()
# toolbox.register("optimizer", random.choice, ["SGD", "Adam", "AdamW"])
# toolbox.register("cos_lr", random.choice, [True, False])
# toolbox.register("rect", random.choice, [True, False])
# toolbox.register("amp", random.choice, [True, False])
# toolbox.register("multi_scale", random.choice, [True, False])
# toolbox.register("lr0", random.uniform, 1e-4, 1e-2)
# toolbox.register("lrf", random.uniform, 0.01, 0.3)
# toolbox.register("momentum", random.uniform, 0.85, 0.99)
# toolbox.register("weight_decay", random.uniform, 1e-5, 1e-3)
# toolbox.register("warmup_epochs", random.uniform, 0, 5)
# toolbox.register("warmup_momentum", random.uniform, 0.7, 0.9)
# toolbox.register("warmup_bias_lr", random.uniform, 0.05, 0.2)
# toolbox.register("box", random.uniform, 0.02, 0.2)
# toolbox.register("cls", random.uniform, 0.2, 4.0)
# toolbox.register("dfl", random.uniform, 0.5, 2.0)
# toolbox.register("pose", random.uniform, 1.0, 20.0)
# toolbox.register("kobj", random.uniform, 0.1, 2.0)
# toolbox.register("label_smoothing", random.uniform, 0.0, 0.1)
# toolbox.register("hsv_h", random.uniform, 0.0, 0.1)
# toolbox.register("hsv_s", random.uniform, 0.5, 1.0)
# toolbox.register("hsv_v", random.uniform, 0.2, 1.0)
# toolbox.register("degrees", random.uniform, 0, 5)
# toolbox.register("translate", random.uniform, 0, 0.5)
# toolbox.register("scale", random.uniform, 0.5, 1.5)
# toolbox.register("shear", random.uniform, 0.0, 5.0)
# toolbox.register("perspective", random.uniform, 0.0, 0.001)
# toolbox.register("flipud", random.uniform, 0.0, 0.5)
# toolbox.register("fliplr", random.uniform, 0.0, 0.5)
# toolbox.register("mosaic", random.uniform, 0.0, 1.0)
# toolbox.register("mixup", random.uniform, 0.0, 1.0)
# toolbox.register("copy_paste", random.uniform, 0.0, 0.5)

# # Algorithme génétique avec tous les paramètres
# toolbox.register("individual", tools.initCycle, creator.Individual,
#                  (toolbox.optimizer, toolbox.cos_lr, toolbox.rect, toolbox.amp, toolbox.multi_scale,
#                   toolbox.lr0, toolbox.lrf, toolbox.momentum, toolbox.weight_decay, toolbox.warmup_epochs,
#                   toolbox.warmup_momentum, toolbox.warmup_bias_lr, toolbox.box, toolbox.cls, toolbox.dfl,
#                   toolbox.pose, toolbox.kobj, toolbox.label_smoothing, toolbox.hsv_h, toolbox.hsv_s,
#                   toolbox.hsv_v, toolbox.degrees, toolbox.translate, toolbox.scale, toolbox.shear,
#                   toolbox.perspective, toolbox.flipud, toolbox.fliplr, toolbox.mosaic, toolbox.mixup, toolbox.copy_paste), n=1)
# toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# toolbox.register("evaluate", evaluate)
# toolbox.register("mate", tools.cxBlend, alpha=0.5)
# toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.2, indpb=0.2)
# toolbox.register("select", tools.selTournament, tournsize=3)

# # Lancer l'algorithme génétique
# def genetic_algorithm():
#     population = toolbox.population(n=10)
#     ngen = 5
#     cxpb, mutpb = 0.5, 0.2
#     for gen in range(ngen):
#         offspring = algorithms.varAnd(population, toolbox, cxpb, mutpb)
#         fits = map(toolbox.evaluate, offspring)
#         for fit, ind in zip(fits, offspring):
#             ind.fitness.values = fit
#         population = toolbox.select(offspring, k=len(population))
#     best_ind = tools.selBest(population, 1)[0]
#     print("Best hyperparameters:", best_ind)

# genetic_algorithm()
